# Wrangle Data

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#gathering">Gathering The Data</a></li>
<li><a href="#access">Accessing The Data</a></li>
<li><a href="#cleaning">Cleaning The Data</a></li>
</ul>

<a id="intro"></a>
## Intro
This dataset reports flights in the United States, including carriers, arrival and departure delays, and reasons for delays, from 2006 to 2008. Variable documentation can be found in: http://stat-computing.org/dataexpo/2009/the-data.html


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

<a id="gathering"></a>
### 1. Gather Data

Declare function:
<ol>
    <li> <a id='get_merge_table'> get_merge_table </a> </li>
    <li> <a id='open_flight_data'> open_flight_data </a> </li>
</ol>

In [2]:
def get_merge_table(path):
    #read and merge csv
    flights = pd.DataFrame()

    #get data from 2006-2008
    for i in range(6,9):
        #declare path
        path = "data/200{}.csv".format(i)

        #declare dataframe to save the data
        data_temp = pd.DataFrame()
        data_temp = pd.read_csv(path, encoding = "ISO-8859-1")

        print("read:", path)
        #add to column flights
        flights = flights.append(data_temp.copy())
        
        #save all data flights (2006-2008)
        flights.to_csv(path, index = False)
    return flights

In [3]:
def open_flight_data(path):
    try:
        df = pd.read_csv(path)
    except: 
        df = get_merge_table(path)
    return df

<a id="access"></a>
### 2. Assess

Use function:
<ol>
    <li> <a href='#open_flight_data'> open_flight_data </a> </li>
</ol>

In [4]:
flights = open_flight_data("data/flights.csv")
flights.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2006,1,11,3,743.0,745,1024.0,1018,US,343,...,45.0,13.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
1,2006,1,11,3,1053.0,1053,1313.0,1318,US,613,...,27.0,19.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
2,2006,1,11,3,1915.0,1915,2110.0,2133,US,617,...,4.0,11.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
3,2006,1,11,3,1753.0,1755,1925.0,1933,US,300,...,16.0,10.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
4,2006,1,11,3,824.0,832,1015.0,1015,US,765,...,27.0,12.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0


In [5]:
# lets see the shape and type data in tables with df.info
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21604865 entries, 0 to 21604864
Data columns (total 29 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtypes: float64(14), int64(10), object(5)
memory usage: 4.7+ GB


Use function:
<ol>
    <li> <a id='get_missing_value_percentage'> get_missing_value_percentage </a> </li>
</ol>

In [6]:
def get_missing_value_percentage(data_frame):
    data_missing = data_frame.isna()
    num_data_missing = data_missing.sum()
    num_data = len(data_frame)
    return (num_data_missing * 100)/num_data

In [7]:
get_missing_value_percentage(flights)

Year                  0.000000
Month                 0.000000
DayofMonth            0.000000
DayOfWeek             0.000000
DepTime               1.939045
CRSDepTime            0.000000
ArrTime               2.164772
CRSArrTime            0.000000
UniqueCarrier         0.000000
FlightNum             0.000000
TailNum               0.385964
ActualElapsedTime     2.178889
CRSElapsedTime        0.008526
AirTime               2.178889
ArrDelay              2.178889
DepDelay              1.939045
Origin                0.000000
Dest                  0.000000
Distance              0.000000
TaxiIn                0.701921
TaxiOut               0.634385
Cancelled             0.000000
CancellationCode     98.055452
Diverted              0.000000
CarrierDelay         25.387768
WeatherDelay         25.387768
NASDelay             25.387768
SecurityDelay        25.387768
LateAircraftDelay    25.387768
dtype: float64

In [8]:
flights.CancellationCode.value_counts()

A    176764
B    154753
C     88392
D       208
Name: CancellationCode, dtype: int64

CancellationCode have large null data maybe because large flight not cancelled, because this is important value, so I decide to not delete them.

In [9]:
flights.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,2.160486e+07,2.160486e+07,2.160486e+07,2.160486e+07,2.118594e+07,2.160486e+07,2.113717e+07,2.160486e+07,2.160486e+07,2.113412e+07,...,2.160486e+07,2.145322e+07,2.146781e+07,2.160486e+07,2.160486e+07,1.611987e+07,1.611987e+07,1.611987e+07,1.611987e+07,1.611987e+07
mean,2.006994e+03,6.485366e+00,1.572814e+01,3.936014e+00,1.337860e+03,1.329679e+03,1.483426e+03,1.495324e+03,2.199371e+03,1.265598e+02,...,7.246490e+02,6.869023e+00,1.616451e+01,1.944544e-02,2.343454e-03,4.783657e+00,9.446234e-01,4.961762e+00,3.179883e-02,6.322693e+00
std,8.093108e-01,3.420013e+00,8.788149e+00,1.990878e+00,4.781764e+02,4.638546e+02,5.044608e+02,4.809798e+02,1.974503e+03,7.090811e+01,...,5.663215e+02,1.842550e+01,1.149309e+01,1.380844e-01,4.835248e-02,2.285184e+01,1.054409e+01,1.846588e+01,1.259984e+00,2.348937e+01
min,2.006000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,...,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.006000e+03,4.000000e+00,8.000000e+00,2.000000e+00,9.300000e+02,9.290000e+02,1.108000e+03,1.115000e+03,5.970000e+02,7.600000e+01,...,3.200000e+02,4.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.007000e+03,6.000000e+00,1.600000e+01,4.000000e+00,1.328000e+03,1.321000e+03,1.514000e+03,1.519000e+03,1.525000e+03,1.080000e+02,...,5.760000e+02,5.000000e+00,1.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.008000e+03,9.000000e+00,2.300000e+01,6.000000e+00,1.731000e+03,1.720000e+03,1.910000e+03,1.906000e+03,3.507000e+03,1.560000e+02,...,9.510000e+02,8.000000e+00,1.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.008000e+03,1.200000e+01,3.100000e+01,7.000000e+00,2.930000e+03,2.359000e+03,2.955000e+03,2.400000e+03,9.743000e+03,1.879000e+03,...,4.962000e+03,1.501000e+03,6.020000e+02,1.000000e+00,1.000000e+00,2.580000e+03,1.429000e+03,1.392000e+03,3.920000e+02,1.366000e+03


In [10]:
flights[flights['DepTime']>2359].DepTime.value_counts().head(7)

2400.0    2148
2405.0      28
2410.0      26
2430.0      24
2420.0      24
2425.0      20
2415.0      20
Name: DepTime, dtype: int64

In [11]:
flights[flights['DepTime'] >2359].sort_values(by='DepTime', ascending=False).head(7)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
5338973,2006,9,24,7,2930.0,2140,2955.0,2233,EV,4101,...,1441.0,6.0,0,NaN,0,442.0,0.0,0.0,0.0,0.0
299844,2006,1,13,5,2755.0,2023,2923.0,2219,EV,4284,...,1443.0,10.0,0,NaN,0,0.0,424.0,0.0,0.0,0.0
2030697,2006,4,21,5,2655.0,2230,2731.0,2310,EV,4588,...,1443.0,9.0,0,NaN,0,261.0,0.0,0.0,0.0,0.0
853461,2006,2,6,1,2644.0,1346,2823.0,1554,EV,4209,...,1445.0,4.0,0,NaN,0,0.0,0.0,749.0,0.0,0.0
5653155,2006,10,16,1,2640.0,2255,2635.0,2245,EV,4356,...,1448.0,17.0,0,NaN,0,225.0,0.0,5.0,0.0,0.0
4442343,2006,8,10,4,2635.0,2127,2717.0,2209,EV,4116,...,1441.0,20.0,0,NaN,0,0.0,308.0,0.0,0.0,0.0
5651529,2006,10,23,1,2633.0,2230,2741.0,2344,EV,4310,...,1445.0,17.0,0,NaN,0,237.0,0.0,0.0,0.0,0.0


In [12]:
# check flight time
flights['AirTime'].describe()

count    2.113412e+07
mean     1.032188e+02
std      7.081935e+01
min     -1.425000e+03
25%      5.500000e+01
50%      8.500000e+01
75%      1.310000e+02
max      1.958000e+03
Name: AirTime, dtype: float64

In [13]:
# check flight time < 0
len(flights.query('AirTime < 0'))

3260

In [14]:
# check delay time
flights['ArrDelay'].describe()

count    2.113412e+07
mean     9.035575e+00
std      3.816833e+01
min     -5.920000e+02
25%     -9.000000e+00
50%     -1.000000e+00
75%      1.300000e+01
max      2.598000e+03
Name: ArrDelay, dtype: float64

In [15]:
# check delay time
flights['DepDelay'].describe()

count    2.118594e+07
mean     1.050373e+01
std      3.502096e+01
min     -1.200000e+03
25%     -4.000000e+00
50%      0.000000e+00
75%      9.000000e+00
max      2.601000e+03
Name: DepDelay, dtype: float64

In [16]:
# check delay time
flights['CarrierDelay'].describe()

count    1.611987e+07
mean     4.783657e+00
std      2.285184e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.580000e+03
Name: CarrierDelay, dtype: float64

In [17]:
# check delay time
flights['WeatherDelay'].describe()

count    1.611987e+07
mean     9.446234e-01
std      1.054409e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.429000e+03
Name: WeatherDelay, dtype: float64

In [18]:
# check delay time
flights['NASDelay'].describe()

count    1.611987e+07
mean     4.961762e+00
std      1.846588e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.392000e+03
Name: NASDelay, dtype: float64

In [19]:
# check delay time
flights['SecurityDelay'].describe()

count    1.611987e+07
mean     3.179883e-02
std      1.259984e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.920000e+02
Name: SecurityDelay, dtype: float64

In [20]:
# check delay time
flights['LateAircraftDelay'].describe()

count    1.611987e+07
mean     6.322693e+00
std      2.348937e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.366000e+03
Name: LateAircraftDelay, dtype: float64

Issue:
1. In the variables documentation, DepTime and CRSDepTime is hhmm, also for arrival time. But, in this table they be the normal int or float. 
2. The data not include different or delay elapsed Time. That different can make easier to analysis.

Assumtion:
1. Time more than 2400 indicated that it is time in AM but in the next day (not the same day).


<a id="cleaning"></a>
### 3. Clean

Todo:
<ol>
    <li> <a href='#c1'> Change actual depature/arrival time, schedule depature/arrival time into time type.</a> </li>
    <li> <a href='#c2'> Add delay ellapsed time to make analtsis easier </a> </li>
</ol>

##### Copy flights into clean flights
it can make easier for undo or redo if something happend

In [21]:
clean_flights = flights.copy()
clean_flights.head(3) 

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2006,1,11,3,743.0,745,1024.0,1018,US,343,...,45.0,13.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
1,2006,1,11,3,1053.0,1053,1313.0,1318,US,613,...,27.0,19.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
2,2006,1,11,3,1915.0,1915,2110.0,2133,US,617,...,4.0,11.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0


<a id="c1"></a>
##### 1. Change depature time and CRS depature time type into Time Format

In [22]:
# departure

# make Departure Time (hhmm) to second (s)
clean_flights['DepTime'] = (pd.to_timedelta(clean_flights.DepTime // 100, unit='h') +\
                   pd.to_timedelta(flights.DepTime % 100, unit='m'))

# make Scheduled Departure (hhmm) Time to second (s)
clean_flights['CRSDepTime'] = (pd.to_timedelta(clean_flights.CRSDepTime // 100, unit='h') +\
                      pd.to_timedelta(clean_flights.CRSDepTime % 100, unit='m'))

In [23]:
# arrive

# make Departure Time (hhmm) to second (s)
clean_flights['ArrTime'] = (pd.to_timedelta(clean_flights.ArrTime // 100, unit='h') +\
                   pd.to_timedelta(flights.ArrTime % 100, unit='m'))

# make Scheduled Departure (hhmm) Time to second (s)
clean_flights['CRSArrTime'] = (pd.to_timedelta(clean_flights.CRSArrTime // 100, unit='h') +\
                      pd.to_timedelta(clean_flights.CRSArrTime % 100, unit='m'))

In [24]:
clean_flights.head(2)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2006,1,11,3,07:43:00,07:45:00,10:24:00,10:18:00,US,343,...,45.0,13.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
1,2006,1,11,3,10:53:00,10:53:00,13:13:00,13:18:00,US,613,...,27.0,19.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0


In [25]:
clean_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21604865 entries, 0 to 21604864
Data columns (total 29 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              timedelta64[ns]
CRSDepTime           timedelta64[ns]
ArrTime              timedelta64[ns]
CRSArrTime           timedelta64[ns]
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtypes: float64(12), int64(8), ob

Now the depature time in second, so type data integer or float are valid.

<a id="c2"></a>
##### 2. Add different ellapsed time to make analtsis easier

In [26]:
clean_flights.insert(14, 'ElapsedDelay', np.nan)
clean_flights['ElapsedDelay'] = (clean_flights['ActualElapsedTime'] - clean_flights['CRSElapsedTime'])
clean_flights.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2006,1,11,3,07:43:00,07:45:00,10:24:00,10:18:00,US,343,...,45.0,13.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
1,2006,1,11,3,10:53:00,10:53:00,13:13:00,13:18:00,US,613,...,27.0,19.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
2,2006,1,11,3,19:15:00,19:15:00,21:10:00,21:33:00,US,617,...,4.0,11.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
3,2006,1,11,3,17:53:00,17:55:00,19:25:00,19:33:00,US,300,...,16.0,10.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0
4,2006,1,11,3,08:24:00,08:32:00,10:15:00,10:15:00,US,765,...,27.0,12.0,0,NaN,0,0.0,0.0,0.0,0.0,0.0


In [27]:
clean_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21604865 entries, 0 to 21604864
Data columns (total 30 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              timedelta64[ns]
CRSDepTime           timedelta64[ns]
ArrTime              timedelta64[ns]
CRSArrTime           timedelta64[ns]
UniqueCarrier        object
FlightNum            int64
TailNum              object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ElapsedDelay         float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             int64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     object
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtyp

## Save Clean Flights

In [28]:
clean_flights.to_csv("data/clean_flights.csv", index = False)